In [1]:
import fxcmpy
import socketio
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sympy import *
init_printing()
import xlrd
import itertools
import datetime
from itertools import combinations
from itertools import product
from itertools import groupby
from astropy.table import QTable, Table, Column
from astropy import units as u
from IPython.display import HTML
import requests
import json

C:\ProgramData\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
TOKEN = 'b199bb5a15fa26957990a3fb823a51bd473cca55'
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error', server='demo', log_file='log.txt')


## WATCHLIST = ['AUD/CAD','AUD/CHF','AUD/JPY','AUD/NZD','AUD/USD','CAD/CHF','CAD/JPY','CHF/JPY','EUR/AUD','EUR/CAD','EUR/CHF','EUR/GBP','EUR/JPY','EUR/NZD','EUR/USD','GBP/AUD','GBP/CAD','GBP/CHF','GBP/JPY','GBP/USD','NZD/CAD','NZD/CHF','NZD/JPY','NZD/USD','USD/CAD','USD/CHF','USD/JPY']

pair = ['AUD/CAD','AUD/CHF','AUD/JPY','AUD/NZD','AUD/USD','CAD/CHF','CAD/JPY','CHF/JPY','EUR/AUD','EUR/CAD','EUR/CHF','EUR/GBP','EUR/JPY','EUR/NZD','EUR/USD','GBP/AUD','GBP/CAD','GBP/CHF','GBP/JPY','GBP/USD','NZD/CAD','NZD/CHF','NZD/JPY','NZD/USD','USD/CAD','USD/CHF','USD/JPY']
GRADE = []
MOVE = []

h = len(pair)
index = np.arange(0,h)
gradetable = pd.DataFrame(index=index, columns=['PAIR','MOVE','GRADE'])

for x in pair:
    S = con.get_candles(x, period='H4', number=1920)          ## SETTING TO 12 DAYS FOR NOW
    S['open'] = (S['bidopen']+S['askopen'])/2
    S['close'] = (S['bidclose']+S['askclose'])/2
    S['high'] = (S['bidhigh']+S['askhigh'])/2
    S['low'] = (S['bidlow']+S['asklow'])/2
    S = S.assign(MOVE=(S['close'] - S['open']))
    AvgCandle = np.average(abs(S['MOVE']))

    s = 96                                      ## NUMBER OF 1-HOUR CANDLESTICKS WE WANT TO FORECAST AHEAD (4 DAYS)

    CP = S['close'].iloc[-1]
    l = int(S['close'].count())
    Sd = np.linspace(1,l,num=l)

    D = con.get_candles(x, period='H4', number=1920)          ## SETTING TO 12 DAYS FOR NOW
    D['open'] = (D['bidopen']+D['askopen'])/2
    D['close'] = (D['bidclose']+D['askclose'])/2
    D['high'] = (D['bidhigh']+D['askhigh'])/2
    D['low'] = (D['bidlow']+D['asklow'])/2
    D = D.assign(MOVE=(D['close'] - D['open']))
    AvgCandleH4 = np.average(abs(D['MOVE']))

    k = int(D['close'].count())
    Sdk = np.linspace(1,k,num=k)

    p = k
    RED = []
    BLUE = []
    GREEN = []
    CLOUDA = []
    CLOUDB = []

    for x in Sdk:
        if p < 9:
            RLhigh = 1
            RLlow = 1
        else:
            RL = D[p-9:p]
            RLhigh = max(RL['high'])
            RLlow = min(RL['low'])
        redline = (RLhigh + RLlow)/2
        RED.append(redline)

        if p < 26:
            BLhigh = 1
            BLlow = 1
        else:
            BL = D[p-26:p]
            BLhigh = max(BL['high'])
            BLlow = min(BL['low'])
        blueline = (BLhigh + BLlow)/2
        BLUE.append(blueline)

        greenline = BL['close'][0]
        GREEN.append(greenline)

        if p < 52:
            CLhigh = 1
            CLlow = 1
        else:
            CL = D[p-52:p]
            CLhigh = max(CL['high'])
            CLlow = min(CL['low'])
        clouda = (redline+ blueline)/2
        cloudb = (CLhigh + CLlow)/2
        CLOUDA.append(clouda)
        CLOUDB.append(cloudb)

        p = p - 1

    CLOUDA.reverse() 
    CLOUDB.reverse() 
    cloud = [i / j for i, j in zip(CLOUDA, CLOUDB)]
    cloud = [x-1 for x in cloud]
    D['Cloud'] = cloud
    pclouda = D['close']/CLOUDA - 1
    D['PvCloudA'] = pclouda
    pcloudb = D['close']/CLOUDB - 1
    D['PvCloudB'] = pcloudb

    RED.reverse()
    BLUE.reverse() 
    redblue = [i / j for i, j in zip(RED, BLUE)]
    redblue = [x-1 for x in redblue]
    D['RedvBlue'] = redblue
    pblue = D['close']/BLUE - 1
    D['PvBlue'] = pblue

    GREEN.reverse() 
    pgreen = D['close']/GREEN - 1
    D['PvGreen'] = pgreen

    D['BLUE'] = BLUE
    D = D.assign(BluevLast=((D['BLUE'] - D['BLUE'].shift())/D['BLUE']))

    tg = int(round(k*(14/16),0))
    trendnow = np.average(D['BluevLast'][tg::])                   ##        CURRENT TREND    !!!


    if trendnow > 0:
        Igrade_cloud = cloud[-1]*100
        Igrade_pclouda = pclouda[-1]*100
        Igrade_pcloudb = pcloudb[-1]*100
        Igrade_redblue = redblue[-1]*100
        Igrade_pblue = pblue[-1]*100
        Igrade_pgreen = pgreen[-1]*100
        Igrade_blue = D['BluevLast'][-1]*100

    elif trendnow < 0:
        Igrade_cloud = cloud[-1]*-100
        Igrade_pclouda = pclouda[-1]*-100
        Igrade_pcloudb = pcloudb[-1]*-100
        Igrade_redblue = redblue[-1]*-100
        Igrade_pblue = pblue[-1]*-100
        Igrade_pgreen = pgreen[-1]*-100
        Igrade_blue = D['BluevLast'][-1]*-100
    
    if Igrade_cloud <= 0:
        Igrade_cloud = 0
    if Igrade_pclouda <= 0:
        Igrade_pclouda = 0
    if Igrade_pcloudb <= 0:
        Igrade_pcloudb = 0
    if Igrade_redblue <= 0:
        Igrade_redblue = 0
    if Igrade_pblue <= 0:
        Igrade_pblue = 0
    if Igrade_pgreen <= 0:
        Igrade_pgreen = 0
    if Igrade_blue <= 0:
        Igrade_blue = 0
    

    K = list(D['Cloud'][k-52::])

    Kgains = []
    Klosses = []
    sum_gains = 0
    sum_losses = 1

    if trendnow > 0:
        for x in K:
            if x > 0:
                Kgains.append(x)
            elif x < 0:
                Klosses.append(x)
    
            sum_gains = np.sum(Kgains)
            sum_losses = abs(np.sum(Klosses))
    elif trendnow < 0:
        for x in K:
            if x < 0:
                Kgains.append(x)
            elif x > 0:
                Klosses.append(x)
    
            sum_gains = abs(np.sum(Kgains))
            sum_losses = abs(np.sum(Klosses))

    Igrade_cloud2 = (sum_gains/(sum_gains+sum_losses))


    K = list(D['PvCloudA'][k-52::])

    Kgains = []
    Klosses = []
    sum_gains = 0
    sum_losses = 1

    if trendnow > 0:
        for x in K:
            if x > 0:
                Kgains.append(x)
            elif x < 0:
                Klosses.append(x)
    
            sum_gains = np.sum(Kgains)
            sum_losses = abs(np.sum(Klosses))
    elif trendnow < 0:
        for x in K:
            if x < 0:
                Kgains.append(x)
            elif x > 0:
                Klosses.append(x)
    
            sum_gains = abs(np.sum(Kgains))
            sum_losses = abs(np.sum(Klosses))
    
    Igrade_pclouda2 = (sum_gains/(sum_gains+sum_losses))


    K = list(D['PvCloudB'][k-52::])

    Kgains = []
    Klosses = []
    sum_gains = 0
    sum_losses = 1

    if trendnow > 0:
        for x in K:
            if x > 0:
                Kgains.append(x)
            elif x < 0:
                Klosses.append(x)

            sum_gains = np.sum(Kgains)
            sum_losses = abs(np.sum(Klosses))
    elif trendnow < 0:
        for x in K:
            if x < 0:
                Kgains.append(x)
            elif x > 0:
                Klosses.append(x)
    
            sum_gains = abs(np.sum(Kgains))
            sum_losses = abs(np.sum(Klosses))
    
    Igrade_pcloudb2 = (sum_gains/(sum_gains+sum_losses))


    K = list(D['RedvBlue'][k-26::])

    Kgains = []
    Klosses = []
    sum_gains = 0
    sum_losses = 1

    if trendnow > 0:
        for x in K:
            if x > 0:
                Kgains.append(x)
            elif x < 0:
                Klosses.append(x)
    
            sum_gains = np.sum(Kgains)
            sum_losses = abs(np.sum(Klosses))
    elif trendnow < 0:
        for x in K:
            if x < 0:
                Kgains.append(x)
            elif x > 0:
                Klosses.append(x)
    
            sum_gains = abs(np.sum(Kgains))
            sum_losses = abs(np.sum(Klosses))
    
    Igrade_redblue2 = (sum_gains/(sum_gains+sum_losses))

    K = list(D['PvBlue'][k-26::])

    Kgains = []
    Klosses = []
    sum_gains = 0
    sum_losses = 1

    if trendnow > 0:
        for x in K:
            if x > 0:
                Kgains.append(x)
            elif x < 0:
                Klosses.append(x)
    
            sum_gains = np.sum(Kgains)
            sum_losses = abs(np.sum(Klosses))
    elif trendnow < 0:
        for x in K:
            if x < 0:
                Kgains.append(x)
            elif x > 0:
                Klosses.append(x)
    
            sum_gains = abs(np.sum(Kgains))
            sum_losses = abs(np.sum(Klosses))
    
    Igrade_pblue2 = (sum_gains/(sum_gains+sum_losses))

    K = list(D['PvGreen'][k-26::])

    Kgains = []
    Klosses = []
    sum_gains = 0
    sum_losses = 1

    if trendnow > 0:
        for x in K:
            if x > 0:
                Kgains.append(x)
            elif x < 0:
                Klosses.append(x)
    
            sum_gains = np.sum(Kgains)
            sum_losses = abs(np.sum(Klosses))
    elif trendnow < 0:
        for x in K:
            if x < 0:
                Kgains.append(x)
            elif x > 0:
                Klosses.append(x)
    
            sum_gains = abs(np.sum(Kgains))
            sum_losses = abs(np.sum(Klosses))
    
    Igrade_pgreen2 = (sum_gains/(sum_gains+sum_losses))

    K = list(D['BluevLast'][k-26::])
    
    Kgains = []
    Klosses = []
    sum_gains = 0
    sum_losses = 1
    
    if trendnow > 0:
        for x in K:
            if x > 0:
                Kgains.append(x)
            elif x < 0:
                Klosses.append(x)
    
            sum_gains = np.sum(Kgains)
            sum_losses = abs(np.sum(Klosses))
    elif trendnow < 0:
        for x in K:
            if x < 0:
                Kgains.append(x)
            elif x > 0:
                Klosses.append(x)
    
            sum_gains = abs(np.sum(Kgains))
            sum_losses = abs(np.sum(Klosses))
    
    Igrade_blue2 = (sum_gains/(sum_gains+sum_losses))
    
    Iw1 = 0.20
    Iw2 = 0.20
    Iw3 = 0.12
    Iw4 = 0.12
    Iw5 = 0.06
    Iw6 = 0.30

    Igrade = Iw1*Igrade_cloud + Iw2*((Igrade_pclouda + Igrade_pcloudb)/2) + Iw3*Igrade_redblue + Iw4*Igrade_pblue + Iw5*Igrade_pgreen + Iw6*Igrade_blue

    Igrade2 = Iw1*Igrade_cloud2 + Iw2*((Igrade_pclouda2 + Igrade_pcloudb2)/2) + Iw3*Igrade_redblue2 + Iw4*Igrade_pblue2 + Iw5*Igrade_pgreen2 + Iw6*Igrade_blue2


    I1w = 0.25
    I2w = 0.75

    Grade = I1w*Igrade + I2w*Igrade2

    if trendnow > 0:
        move = 'rise'
    else:
        move = 'fall'

    GRADE.append(Grade)
    MOVE.append(move)
    
pair = list(pair)
GRADE = list(GRADE)
MOVE = list(MOVE)

gradetable = gradetable.assign(PAIR=pair)
gradetable = gradetable.assign(MOVE=MOVE)
gradetable = gradetable.assign(GRADE=GRADE)

gradetable = gradetable.sort_values(['GRADE'], ascending=False)
gradetable.to_excel('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Monitor/ScreenerIchimoku/ScreenerIchimoku.xlsx')

G = zip(pair,MOVE,GRADE)
G = list(G)

[('AUD/CAD', 'rise', 0.851243217888361), ('AUD/CHF', 'fall', 0.05858926987984353), ('AUD/JPY', 'fall', 0.12321959329690439), ('AUD/NZD', 'rise', 0.8165783197076983), ('AUD/USD', 'fall', 0.10579626846504556), ('CAD/CHF', 'fall', 0.28564668369715546), ('CAD/JPY', 'fall', 0.4944750210278716), ('CHF/JPY', 'fall', 0.8246640824155753), ('EUR/AUD', 'rise', 0.03552092047786026), ('EUR/CAD', 'rise', 0.26142462974848235), ('EUR/CHF', 'fall', 0.33359747345844776), ('EUR/GBP', 'rise', 0.26491349020914967), ('EUR/JPY', 'fall', 0.7791329945071681), ('EUR/NZD', 'rise', 0.28450482966993373), ('EUR/USD', 'fall', 0.6893175718094081), ('GBP/AUD', 'fall', 0.9243563073316228), ('GBP/CAD', 'rise', 0.17506982458569875), ('GBP/CHF', 'fall', 0.255368112586032), ('GBP/JPY', 'fall', 0.6918011610090914), ('GBP/USD', 'fall', 0.5888788507977514), ('NZD/CAD', 'rise', 0.49091385890959943), ('NZD/CHF', 'fall', 0.2770530966692946), ('NZD/JPY', 'fall', 0.6307308868360598), ('NZD/USD', 'fall', 0.5129477173291019), ('USD/